# Лабораторная работа 1: Проведение исследований с алгоритмом KNN
**Наборы данных:**
- Классификация: Bookstore Financial Dataset (Calgary) — Kaggle dataset by Gabrielle Charlton.
- Регрессия: Market Trend & External Factors Dataset — Kaggle dataset by Kundan Bedmutha.

**Инструкции по данным:**
1. Скачайте нужные CSV-файлы с Kaggle и положите их в папку `data/` рядом с этим ноутбуком.
   - Рекомендуемые имена файлов:
     - `data/bookstore.csv` — данные для классификации (Bookstore Financial Dataset). Подробнее: https://www.kaggle.com/datasets/gabriellecharlton/bookstore-financial-dataset-2019-2024-calgary. citeturn0search0
     - `data/market_trend.csv` — данные для регрессии (Market Trend & External Factors). Подробнее: https://www.kaggle.com/datasets/kundanbedmutha/market-trend-and-external-factors-dataset. citeturn0search1
2. Если вы используете Kaggle API, положите `kaggle.json` в `~/.kaggle/` и используйте `kaggle datasets download`.

**Задание по ноутбуку соответствует документу с лабораторными:** см. загруженный файл `Лабы_фреймворки.docx`. fileciteturn0file0

---


In [4]:
# Импорт стандартных библиотек и вспомогательных функций
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
# Импорт моделей sklearn
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor

RND = 42


In [5]:
# Загрузка данных
bookstore_path = 'data/bookstore.csv'   # классификация
market_path = 'data/market_trend.csv'   # регрессия

# Попытка загрузки — если файла нет, появится понятная ошибка.
try:
    df_book = pd.read_csv(bookstore_path)
    print('Bookstore dataset loaded, shape:', df_book.shape)
except Exception as e:
    print('Не удалось загрузить bookstore.csv — проверьте путь. Ошибка:', e)
    df_book = None

try:
    df_market = pd.read_csv(market_path)
    print('Market dataset loaded, shape:', df_market.shape)
except Exception as e:
    print('Не удалось загрузить market_trend.csv — проверьте путь. Ошибка:', e)
    df_market = None


Bookstore dataset loaded, shape: (21566, 10)
Market dataset loaded, shape: (30000, 14)


In [6]:
# Быстрая разведочная аналитика (EDA)
# Смотрим первые строки, пропуски и типы признаков.
if df_book is not None:
    display(df_book.head())
    print('\nBookstore info:')
    print(df_book.info())
    print('\nMissing values (book):\n', df_book.isnull().sum().sort_values(ascending=False).head(10))

if df_market is not None:
    display(df_market.head())
    print('\nMarket info:')
    print(df_market.info())
    print('\nMissing values (market):\n', df_market.isnull().sum().sort_values(ascending=False).head(10))


,month,from,to,sku,qty,unit_cost,unit_price,extended_cost,extended_retail,dataset
0,2019-01-01,YYC-WH,YYC-DT,BK1000,19,8.09,18.92,153.71,359.48,bookstore_inventory
1,2019-01-01,YYC-WH,YYC-NW,BK1000,10,8.09,18.92,80.90,189.20,bookstore_inventory
2,2019-01-01,YYC-WH,YYC-SE,BK1000,7,8.09,18.92,56.63,132.44,bookstore_inventory
3,2019-01-01,YYC-WH,YYC-DT,BK1001,21,10.75,24.49,225.75,514.29,bookstore_inventory
4,2019-01-01,YYC-WH,YYC-NW,BK1001,13,10.75,24.49,139.75,318.37,bookstore_inventory



Bookstore info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21566 entries, 0 to 21565
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   month            21566 non-null  object 
 1   from             21566 non-null  object 
 2   to               21566 non-null  object 
 3   sku              21566 non-null  object 
 4   qty              21566 non-null  int64  
 5   unit_cost        21566 non-null  float64
 6   unit_price       21566 non-null  float64
 7   extended_cost    21566 non-null  float64
 8   extended_retail  21566 non-null  float64
 9   dataset          21566 non-null  object 
dtypes: float64(4), int64(1), object(5)
memory usage: 1.6+ MB
None

Missing values (book):
 month              0
from               0
to                 0
sku                0
qty                0
unit_cost          0
unit_price         0
extended_cost      0
extended_retail    0
dataset            0
dtype: int64


,Date,Open_Price,Close_Price,High_Price,Low_Price,Volume,Daily_Return_Pct,Volatility_Range,VIX_Close,Economic_News_Flag,Sentiment_Score,Federal_Rate_Change_Flag,GeoPolitical_Risk_Score,Currency_Index
0,1902-09-08,100.00,100.50,100.63,99.35,2334489,0.0000,1.28,31.44,0,-0.413,0,61.60,98.88
1,1902-09-09,100.50,102.02,102.30,99.49,10626850,1.5124,2.81,27.99,1,-0.384,1,69.49,93.43
2,1902-09-10,102.02,101.55,102.56,101.09,9884633,-0.4607,1.47,21.27,1,0.591,0,67.41,84.25
3,1902-09-11,101.55,101.08,104.16,100.13,9405648,-0.4628,4.03,48.86,1,0.599,1,50.91,87.78
4,1902-09-12,101.08,98.65,101.69,98.39,5247581,-2.4040,3.30,15.78,1,-0.081,1,23.00,82.11



Market info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 14 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Date                      30000 non-null  object 
 1   Open_Price                30000 non-null  float64
 2   Close_Price               30000 non-null  float64
 3   High_Price                30000 non-null  float64
 4   Low_Price                 30000 non-null  float64
 5   Volume                    30000 non-null  int64  
 6   Daily_Return_Pct          30000 non-null  float64
 7   Volatility_Range          30000 non-null  float64
 8   VIX_Close                 30000 non-null  float64
 9   Economic_News_Flag        30000 non-null  int64  
 10  Sentiment_Score           30000 non-null  float64
 11  Federal_Rate_Change_Flag  30000 non-null  int64  
 12  GeoPolitical_Risk_Score   30000 non-null  float64
 13  Currency_Index            30000 non-null  float

In [7]:
# Подготовка признаков и baseline модели (KNN)
# Здесь мы формируем простой baseline — минимальная предобработка + модель sklearn.
from sklearn.model_selection import train_test_split

# ----- Классификация (Bookstore) -----
if df_book is not None:
    # Простейшая задача классификации: если в датасете есть колонка 'Type' или 'Category' — используем её как целевую.
    # Если нет — пользователь должен заменить TARGET_BOOK на актуальное имя целевой колонки.
    TARGET_BOOK = 'target'  # <- Поменяйте на фактическое имя столбца с меткой (например 'IsProfitable' или 'Category')
    if TARGET_BOOK not in df_book.columns:
        print('В dataframe bookstore отсутствует столбец', TARGET_BOOK, '. Пожалуйста, замените TARGET_BOOK на нужный столбец.')
    else:
        Xb = df_book.drop(columns=[TARGET_BOOK])
        yb = df_book[TARGET_BOOK]
        # простая обработка числовых признаков: заполняем медианой
        Xb = Xb.select_dtypes(include=[np.number]).fillna(Xb.median())
        # train/test split
        Xb_train, Xb_test, yb_train, yb_test = train_test_split(Xb, yb, test_size=0.2, random_state=RND, stratify=yb if yb.nunique()>1 else None)
        # StandardScaler + модель
        scaler = StandardScaler()
        Xb_train_s = scaler.fit_transform(Xb_train)
        Xb_test_s = scaler.transform(Xb_test)

        # KNN baseline (classification)
        clf = KNeighborsClassifier(n_neighbors=5)
        clf.fit(Xb_train_s, yb_train)
        yb_pred = clf.predict(Xb_test_s)
        print('Classification accuracy (baseline):', accuracy_score(yb_test, yb_pred))

# ----- Регрессия (Market Trend) -----
if df_market is not None:
    # Предполагаем, что в датасете есть столбец 'Target' — замените при необходимости.
    TARGET_MKT = 'target'  # <- замените на имя целевого столбца (например 'Close' или 'NextDayReturn')
    if TARGET_MKT not in df_market.columns:
        print('В dataframe market отсутствует столбец', TARGET_MKT, '. Пожалуйста, замените TARGET_MKT на нужный столбец.')
    else:
        Xm = df_market.drop(columns=[TARGET_MKT])
        ym = df_market[TARGET_MKT]
        # используем числовые признаки только
        Xm = Xm.select_dtypes(include=[np.number]).fillna(Xm.median())
        Xm_train, Xm_test, ym_train, ym_test = train_test_split(Xm, ym, test_size=0.2, random_state=RND)
        scaler_m = StandardScaler()
        Xm_train_s = scaler_m.fit_transform(Xm_train)
        Xm_test_s = scaler_m.transform(Xm_test)

        # KNN regressor baseline
        reg = KNeighborsRegressor(n_neighbors=5)
        reg.fit(Xm_train_s, ym_train)
        ym_pred = reg.predict(Xm_test_s)
        print('Regression RMSE (baseline):', mean_squared_error(ym_test, ym_pred, squared=False))


В dataframe bookstore отсутствует столбец target . Пожалуйста, замените TARGET_BOOK на нужный столбец.
В dataframe market отсутствует столбец target . Пожалуйста, замените TARGET_MKT на нужный столбец.


In [ ]:
# Улучшение бейзлайна — пример гипотез и проверка (предобработка + подбор гиперпараметров)
# Комментарий: здесь описаны гипотезы — масштабирование, выбор числа соседей / глубины деревьев и т.д.
# Пример для классификации: GridSearch по n_neighbors (если KNN) / max_depth (если дерево)
from sklearn.model_selection import GridSearchCV

# Пример общего GridSearch для классификации (если целевой столбец указан)
if df_book is not None and TARGET_BOOK in df_book.columns:
    print('\nЗапускаем GridSearch для классификации (примерные параметры)...')
    param_grid = {'n_neighbors':[3,5,7]} if 'KNN'=='KNN' else {{'max_depth':[3,5,7,10]}}
    try:
        model_for_gs = KNeighborsClassifier() if 'KNN'=='KNN' else (DecisionTreeClassifier(random_state=RND))
        gs = GridSearchCV(model_for_gs, param_grid, cv=5, scoring='accuracy', n_jobs=1)
        gs.fit(Xb_train_s, yb_train)
        print('Best params:', gs.best_params_)
        print('Best CV score:', gs.best_score_)
    except Exception as e:
        print('GridSearch skipped (ошибка или несоответствие модели):', e)


In [ ]:
# Имплементация KNN c нуля (классификация и регрессия)
# Комментарий: простая реализация без оптимизаций — для учебных целей.
from collections import Counter
import numpy as np

class SimpleKNN:
    def __init__(self, k=5, task='classification'):
        self.k = k
        self.task = task

    def fit(self, X, y):
        self.X = np.array(X)
        self.y = np.array(y)

    def _dist(self, a, b):
        return np.sqrt(((a-b)**2).sum(axis=1))

    def predict(self, X):
        X = np.array(X)
        preds = []
        for x in X:
            d = self._dist(self.X, x)
            idx = np.argsort(d)[:self.k]
            neigh = self.y[idx]
            if self.task=='classification':
                most = Counter(neigh).most_common(1)[0][0]
                preds.append(most)
            else:
                preds.append(neigh.mean())
        return np.array(preds)

# Протестируем реализатор на маленьком поднаборе (если данные загружены)
if df_book is not None and TARGET_BOOK in df_book.columns:
    knn = SimpleKNN(k=5, task='classification')
    knn.fit(Xb_train_s, yb_train)
    print('SimpleKNN classification example done. Sample predictions:', knn.predict(Xb_test_s[:5]))

if df_market is not None and TARGET_MKT in df_market.columns:
    knn_r = SimpleKNN(k=5, task='regression')
    knn_r.fit(Xm_train_s, ym_train)
    print('SimpleKNN regression example done. Sample preds:', knn_r.predict(Xm_test_s[:5]))


In [ ]:
# Выводы и дальнейшие шаги
# Комментарий: в этой ячейке опишите полученные результаты, что сработало, какие гипотезы дали улучшение,
# и какие техники вы могли бы добавить (feature engineering, time-series specific processing, target encoding и т.д.).
print('Заполните этот раздел вашими выводами после запуска ноутбука и проверки результатов.')